# Task 3 提示词工程与RAG （选做）

## 前置知识

### **大语言模型与[提示词工程](https://www.promptingguide.ai/zh):**
	
- 理解提示词工程如何引导LLM完成特定任务、生成特定格式输出的技术。
- 了解零样本（Zero-shot）、少样本（Few-shot）和思维链（Chain-of-Thought）等主流提示词策略。



设计有效的提示词以指导模型执行期望任务的方法被称为提示工程。  
零样本提示，即直接提示模型给出一个回答，而没有提供任何示例。类似地有少样本提示。  
思维链通过展示中间推理步骤以实现复杂的推理能力。可以与少样本提示结合起来。  
零样本提示的思维链，大致就是对模型说“让我们一步一步来思考这个问题”。

### [API](./API.ipynb)调用 

- 熟悉如何通过API接口与大语言模型进行交互。


----

### 检索增强生成 (Retrieval-Augmented Generation - [RAG](./Retrieval-Augmented%20Generation%20for%20Knowledge-Intensive%20NLP%20Tasks.md)

- 理解RAG如何帮助大模型解决问题。
- 掌握基础RAG实现的流程（文档加载、文本分割、构建向量数据库、向量数据库检索、结合检索结果生成回答）。

LLM 原始训练数据集之外的新数据称为外部数据。词嵌入将数据存储在向量数据库中。
RAG 将用户查询向量与向量数据库匹配，返回高度相关的特定文档。
RAG 模型通过在上下文中添加检索到的相关数据来增强用户输入（或提示）
需要实时/定期更新外部数据

----


## 数据集

使用学校的研究生手册作为[数据集](https://gs.hust.edu.cn/info/1137/6338.htm)，完成任务。

## 实践任务

1. 提示词构建问答数据集
	- 基于提示词从特定文档中自动构建问答（QA）数据集。
	- 设计提示词以明确大模型的任务边界以及输出格式的约束；
	- 理解长文本分割的必要性及策略，以适配大模型的上下文长度限制；
	- 工具支持：可使用阿里云提供的免费 API 调用大模型或者私戳出题人提供。
	
2. RAG知识问答
	- 理解检索增强生成（RAG）技术在知识问答中的应用原理与核心环节相关知识。
	
	- 使用API以及研究生手册文档或自主构建的QA数据集实现RAG 的基本流程，使其可以对研究生手册或QA数据集进行知识问答。

In [5]:
# 构建问答数据集
import os
import pdfplumber
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
import concurrent.futures
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
import numpy as np
import faiss
import torch

load_dotenv()

def extract_pdf(pdf_path):
    text = ""
    # with as 是 Python 的上下文管理语法，能确保代码块结束后自动关闭文件，避免资源泄漏。
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            # 如果没有文本内容，则用空字符串代替，避免拼接 None 报错。
            text += (page.extract_text() or "") + "\n"
    return text

def extract_chunks(text, max_length=800):
    # 按换行符 \n 分割成一个字符串列表
    paragraphs = text.split('\n')
    chunks, chunk = [], ""
    for para in paragraphs:
        if len(chunk) + len(para) < max_length:
            chunk += para + "\n"
        else:
            chunks.append(chunk.strip())
            chunk = para + "\n"
    if chunk: chunks.append(chunk.strip())  # 最后剩余的内容
    return chunks

def call_llm(system_prompt, content_prompt):
    try:
        client = OpenAI(
            api_key=os.getenv("DASHSCOPE_API_KEY"),
            base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
        )
        completion = client.chat.completions.create(
            model="qwen-plus",
            messages=[
                {'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': content_prompt},
            ]
        )
        return completion.choices[0].message.content
    except Exception as e:
        print(f"错误信息：{e}")
        return ""

def extract_qa(text):
    qa_list, q, a = [], None, None
    for line in text.strip().split('\n'):
        if line.startswith("问题："):
            q = line.replace("问题：", "").strip()  # 去掉开头的 "问题："
        elif line.startswith("答案："):
            a = line.replace("答案：", "").strip()
            if q and a: qa_list.append({"question": q, "answer": a})
            q, a = None, None
    return qa_list

def process_chunk(chunk):
    content_prompt = (
        "你是专业的问答数据集生成助手。请仔细阅读下方内容，尽可能多地生成高质量的问答对。\n"
        "要求：\n"
        "1. 每个问答对要有实际意义，问题要具体，答案要准确。不要问与页码有关的问题。\n"
        "2. 输出格式严格为：问题：...，答案：...\n"
        "3. 不要编造内容，答案必须来自给定内容。\n"
        "4. 自主根据内容确定生成的问答组数。\n"
        "5. 问题和答案之间用换行分隔，不要有多余解释。\n"
        f"内容：{chunk}"
    )
    result = call_llm(system_prompt, content_prompt)
    return extract_qa(result)

system_prompt = '你是一个专业的问答（QA）数据集生成助手，请根据输入内容生成3组问题和答案，格式严格按照：问题：...，答案：...'
pdf_path = "./2024研究生手册.pdf"
text = extract_pdf(pdf_path)
chunks = extract_chunks(text)
qa_list = []
# 创建一个线程池，最多同时运行 5 个线程
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # 用线程池并发地为每个分块提交一个任务，并把所有任务的“未来对象”收集到 futures 列表中。
    futures = [executor.submit(process_chunk, chunk) for chunk in chunks]
    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="分块处理进度"):
        qa = future.result()
        qa_list.extend(qa)
df = pd.DataFrame(qa_list)
df.to_csv("qa_dataset.csv", index=False)
print(df.head())


tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

def get_embedding(text):
    # tokenizer 返回 PyTorch 张量
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
    with torch.no_grad(): # 取第一个 token 的向量，去掉 batch 维度（因为输入是单条文本）
        return model(**inputs).last_hidden_state[:, 0, :].squeeze().cpu().numpy()

docs = df['answer'].tolist()
doc_embeddings = np.vstack([get_embedding(doc) for doc in docs]).astype('float32') # 按行拼接
# [回答数, 向量维度]

# 用 FAISS 库创建一个基于L2距离的扁平向量索引对象，用于后续的相似度检索。
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
# 将所有文档的语义向量（doc_embeddings）添加到 FAISS 索引中，建立向量数据库。
index.add(doc_embeddings)

# 检索函数，用于从向量数据库中找出与查询最相似的文档
def retrieve(query, top_k=10):
    query_vec = get_embedding(query).reshape(1, -1) # -1 表示自动推断剩余的维度，search() 要求输入是二维数组
    _, I = index.search(query_vec, top_k)  # 返回距离和索引 [查询数, top_k]
    return [docs[i] for i in I[0]]

# RAG（检索增强生成）问答函数
system_prompt = '你是华中科技大学研究生手册的智能问答助手。请根据下方提供的内容，准确、简明地回答用户的问题。如果内容中没有相关信息，请直接回复“手册未包含相关内容”。'
def rag_answer(query):
    context = "\n".join(retrieve(query))
    content_prompt = f"回答以下问题：{query}\n可参考手册内容：{context}"
    return call_llm(system_prompt, content_prompt)


print(rag_answer("如何申请学位论文答辩？"))

分块处理进度: 100%|██████████| 178/178 [07:09<00:00,  2.41s/it]


                        question  \
0  华中科技大学博士研究生培养工作规定是经哪次会议审议通过的？   
1       华中科技大学博士研究生培养工作规定的文号是什么？   
2     制定华中科技大学博士研究生培养工作规定的依据有哪些？   
3      华中科技大学博士研究生培养工作规定适用于哪些学生？   
4        华中科技大学博士研究生的培养目标包括哪些方面？   

                                              answer  
0           华中科技大学博士研究生培养工作规定是经2023年8月28日校长办公会审议通过的。  
1                   华中科技大学博士研究生培养工作规定的文号是校研〔2023〕5号。  
2  制定华中科技大学博士研究生培养工作规定的依据是《中华人民共和国学位条例暂行实施办法》（国发〔...  
3                            该规定适用于具有华中科技大学学籍的各类博士生。  
4                      培养博士生成为德智体美劳全面发展的社会主义建设者和接班人。  
申请学位论文答辩的流程如下：

1. 完成课程学习和学位论文。
2. 接受资格审查，通过后方可进入下一环节。
3. 通过学位论文检测。
4. 提交学位申请材料。
5. 学位论文评审通过。
6. 通过上述环节后，方可申请学位论文答辩。

注意：若为研究生结业后申请学位，须从预答辩环节开始；涉密论文需经研究生院审定通过后方可撰写；如需重新提交论文，应向研究生院学位办公室重新提交进行检测。所有答辩材料需与本表一并存档。


FAISS（Facebook AI Similarity Search）是由 Facebook AI Research 开发的高效相似性搜索库，主要用于大规模向量检索和聚类。它支持在 CPU 和 GPU 上进行高维向量的快速相似度计算，常用于文本、图像等嵌入向量的最近邻检索（如 RAG、推荐系统、语义搜索等场景）。FAISS 提供多种索引结构，能处理百万级甚至更大规模的数据集。

In [7]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import numpy as np
import faiss
import torch
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

df = pd.read_csv("qa_dataset.csv")

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        return model(**inputs).last_hidden_state[:, 0, :].squeeze().cpu().numpy()

docs = df['answer'].tolist()
doc_embeddings = np.vstack([get_embedding(doc) for doc in docs]).astype('float32')

index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)

def retrieve(query, top_k=10):
    query_vec = get_embedding(query).reshape(1, -1)
    _, I = index.search(query_vec, top_k)
    return [docs[i] for i in I[0]]

system_prompt = (
    "你是华中科技大学研究生手册的智能问答助手。请结合下方内容，尽量准确、简明地回答用户的问题。"
    "如果确实无法从内容中找到答案，回复“手册未包含相关内容”。"
)

def call_llm(system_prompt, content_prompt):
    try:
        client = OpenAI(
            api_key=os.getenv("DASHSCOPE_API_KEY"),
            base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
        )
        completion = client.chat.completions.create(
            model="qwen-plus",
            messages=[
                {'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': content_prompt},
            ]
        )
        return completion.choices[0].message.content
    except Exception as e:
        print(f"错误信息：{e}")
        return ""

def rag_answer(query):
    context = "\n".join(retrieve(query))
    print("检索到的内容：", context)
    print("现在开始回答：")
    content_prompt = f"回答以下问题：{query}\n可参考手册内容：{context}"
    return call_llm(system_prompt, content_prompt)


print(rag_answer("最长修业年限是多少？"))

检索到的内容： 应在全日制硕士最长学习年限内（含休学）完成学业。
全日制硕士研究生的最长学习年限为4年（含休学）。
2019级及以前的博士生最长学习年限为8年（含休学）。
自2020级开始，直博生和专业学位博士生的最长学习年限为7年（含休学）。
非全日制硕士研究生的最长学习年限为5年（含休学）。
自2020级开始，其他博士生的最长学习年限为6年（含休学）。
应限定在最长学习年限内。
计入我校读研的学习年限。
延期后的毕业时间不得超过相应的最长学习年限（含休学）。
办理结业最迟不超过学校规定的最长学习年限，逾期未办理的应予退学处理。
现在开始回答：
最长修业年限根据学生类型不同而有所区别：

- 全日制硕士研究生：4年（含休学）
- 非全日制硕士研究生：5年（含休学）
- 2019级及以前的博士生：8年（含休学）
- 自2020级开始的直博生和专业学位博士生：7年（含休学）
- 自2020级开始的其他博士生：6年（含休学）

以上均为最长学习年限，含休学时间。
